<h1>Heritage Health</h1>

In [75]:
#Importing libraries

import numpy as np
import pandas as pd

<h2>Data Processing</h2>

We are gonna take first take a look at the claims table and do some preprocessing.

In [76]:
claims.head()

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,1,11,4,28,0.0,9.0,27,0,2,0
1,97903248,3316066.0,726296.0,5300.0,3,4,4,50,0.0,8.0,27,2,1,0
2,2759427,2997752.0,140343.0,91972.0,3,4,4,14,0.0,1.0,22,0,1,0
3,73570559,7053364.0,240043.0,70119.0,3,5,2,24,0.0,6.0,22,2,6,0
4,11837054,7557061.0,496247.0,68968.0,2,11,6,27,0.0,5.0,10,2,1,0


In [77]:
claims.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 845095 entries, 0 to 2668983
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   MemberID               845095 non-null  int64  
 1   ProviderID             840767 non-null  float64
 2   Vendor                 839001 non-null  float64
 3   PCP                    843014 non-null  float64
 4   Year                   845095 non-null  int64  
 5   Specialty              845095 non-null  int64  
 6   PlaceSvc               845095 non-null  int64  
 7   PayDelay               845095 non-null  int64  
 8   LengthOfStay           845095 non-null  float64
 9   DSFS                   845095 non-null  float64
 10  PrimaryConditionGroup  845095 non-null  int64  
 11  CharlsonIndex          845095 non-null  int64  
 12  ProcedureGroup         845095 non-null  int64  
 13  SupLOS                 845095 non-null  int64  
dtypes: float64(5), int64(9)
memory usag

In [78]:
claims["LengthOfStay"].unique()

array([  0.,   1.,  28.,   2.,   3.,   4.,  14.,  56.,   6.,   5., 182.])

Converting every non-numeric & range variable to numeric:

In [79]:
year = {"Y1": 1, "Y2": 2, "Y3": 3}

claims["Year"] = claims["Year"].replace(to_replace=year)

/var/folders/5z/7pdvd1hj33j1wnkkwcw7lvpc0000gn/T/ipykernel_21468/2662217585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims["Year"] = claims["Year"].replace(to_replace=year)


In [80]:
dfds = {"0- 1 month": 1, "1- 2 months": 2, "2- 3 months": 3, "3- 4 months": 4, "4- 5 months": 5, "5- 6 months": 6, 
        "6- 7 months": 7, "7- 8 months": 8, "8- 9 months": 9, "9- 10 months": 10,"9-10 months": 10, "10- 11 months": 11,
        "10-11 months": 11, "11- 12 months": 12, "11-12 months": 12}

claims["DSFS"] = claims["DSFS"].replace(to_replace=dfds)
claims["DSFS"] = claims["DSFS"].fillna(0)

/var/folders/5z/7pdvd1hj33j1wnkkwcw7lvpc0000gn/T/ipykernel_21468/555342680.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims["DSFS"] = claims["DSFS"].replace(to_replace=dfds)
/var/folders/5z/7pdvd1hj33j1wnkkwcw7lvpc0000gn/T/ipykernel_21468/555342680.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims["DSFS"] = claims["DSFS"].fillna(0)


In [81]:
los = {"1 day": 1, "2 days": 2, "3 days": 3, "4 days": 4, "5 days": 5, "6 days": 6, "1- 2 weeks": 14, 
       "2- 4 weeks": 28, "4- 8 weeks": 56, "26+ weeks": 182}

claims["LengthOfStay"] = claims["LengthOfStay"].replace(to_replace=los)
claims["LengthOfStay"] = claims["LengthOfStay"].fillna(0)

/var/folders/5z/7pdvd1hj33j1wnkkwcw7lvpc0000gn/T/ipykernel_21468/2445629506.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims["LengthOfStay"] = claims["LengthOfStay"].replace(to_replace=los)
/var/folders/5z/7pdvd1hj33j1wnkkwcw7lvpc0000gn/T/ipykernel_21468/2445629506.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims["LengthOfStay"] = claims["LengthOfStay"].fillna(0)


In [82]:
ci = {"0": 0, "1-2": 2, "3-4": 4, "5+": 5}

claims["CharlsonIndex"] = claims["CharlsonIndex"].replace(to_replace=ci)

/var/folders/5z/7pdvd1hj33j1wnkkwcw7lvpc0000gn/T/ipykernel_21468/3271690243.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims["CharlsonIndex"] = claims["CharlsonIndex"].replace(to_replace=ci)


In [83]:
claims["PayDelay"] = claims["PayDelay"].replace(to_replace={"162+": 162})
claims["PayDelay"] = pd.to_numeric(claims["PayDelay"])

claims

/var/folders/5z/7pdvd1hj33j1wnkkwcw7lvpc0000gn/T/ipykernel_21468/3207520773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims["PayDelay"] = claims["PayDelay"].replace(to_replace={"162+": 162})
/var/folders/5z/7pdvd1hj33j1wnkkwcw7lvpc0000gn/T/ipykernel_21468/3207520773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims["PayDelay"] = pd.to_numeric(claims["PayDelay"])


,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,1,11,4,28,0.0,9.0,27,0,2,0
1,97903248,3316066.0,726296.0,5300.0,3,4,4,50,0.0,8.0,27,2,1,0
2,2759427,2997752.0,140343.0,91972.0,3,4,4,14,0.0,1.0,22,0,1,0
3,73570559,7053364.0,240043.0,70119.0,3,5,2,24,0.0,6.0,22,2,6,0
4,11837054,7557061.0,496247.0,68968.0,2,11,6,27,0.0,5.0,10,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2668957,65671957,8667706.0,41541.0,58880.0,2,11,4,30,0.0,5.0,37,2,1,0
2668960,34169043,9484587.0,140343.0,16757.0,2,4,4,57,0.0,1.0,19,0,1,0
2668961,46449277,822173.0,917318.0,27406.0,3,4,4,20,0.0,2.0,41,0,1,0
2668967,66218307,6164824.0,791272.0,47987.0,2,4,4,150,0.0,6.0,41,0,2,0


In the cell below we are checking where we could find missing values and discuss how we could deal with missing values.

In [84]:
claims.isna().sum()

MemberID                    0
ProviderID               4328
Vendor                   6094
PCP                      2081
Year                        0
Specialty                   0
PlaceSvc                    0
PayDelay                    0
LengthOfStay                0
DSFS                        0
PrimaryConditionGroup       0
CharlsonIndex               0
ProcedureGroup              0
SupLOS                      0
dtype: int64

The following variables are categorical with several distinct categories:
- Specialty              
- PlaceSvc               
- PrimaryConditionGroup
- ProcedureGroup        

In [85]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# list of columns to be encoded
columns_to_encode = ["Specialty", "PlaceSvc", "PrimaryConditionGroup", "ProcedureGroup"]

# UN-COMMENT FOR ONE-HOT

# # instantiate the one-hot encoder
# ohe = OneHotEncoder()

# # fit and transform the one-hot encoder on the selected columns
# encoded_df = ohe.fit_transform(df[columns_to_encode]).toarray()

# # create a new dataframe with the encoded columns
# encoded_df = pd.DataFrame(encoded_df, columns=ohe.get_feature_names(columns_to_encode))

# # concatenate the original dataframe with the encoded dataframe
# claims = pd.concat([claims.drop(columns_to_encode, axis=1), encoded_df], axis=1)

# UN-COMMENT FOR LABEL
# instantiate the label encoder
le = LabelEncoder()

# fit and transform the label encoder on the selected columns
claims[columns_to_encode] = claims[columns_to_encode].apply(lambda col: le.fit_transform(col))

/var/folders/5z/7pdvd1hj33j1wnkkwcw7lvpc0000gn/T/ipykernel_21468/2162475107.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims[columns_to_encode] = claims[columns_to_encode].apply(lambda col: le.fit_transform(col))


In [86]:
# Dropping unnecessary columns
#claims = claims.drop("ProviderID", axis=1)
# claims = claims.drop("MemberID", axis=1)
claims

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,1,11,4,28,0.0,9.0,27,0,2,0
1,97903248,3316066.0,726296.0,5300.0,3,4,4,50,0.0,8.0,27,2,1,0
2,2759427,2997752.0,140343.0,91972.0,3,4,4,14,0.0,1.0,22,0,1,0
3,73570559,7053364.0,240043.0,70119.0,3,5,2,24,0.0,6.0,22,2,6,0
4,11837054,7557061.0,496247.0,68968.0,2,11,6,27,0.0,5.0,10,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2668957,65671957,8667706.0,41541.0,58880.0,2,11,4,30,0.0,5.0,37,2,1,0
2668960,34169043,9484587.0,140343.0,16757.0,2,4,4,57,0.0,1.0,19,0,1,0
2668961,46449277,822173.0,917318.0,27406.0,3,4,4,20,0.0,2.0,41,0,1,0
2668967,66218307,6164824.0,791272.0,47987.0,2,4,4,150,0.0,6.0,41,0,2,0


In [87]:
duplicateRows = claims[claims.duplicated(['MemberID'])]

duplicateRowsMember = claims.loc[claims['MemberID'] == 58458320, :]

claims = claims.drop_duplicates(subset= ['MemberID', 'Year', 'PrimaryConditionGroup'])

claims_agg = claims.groupby(['Year', 'MemberID']).agg({
    'ProviderID': ['count', 'nunique'],
    'PrimaryConditionGroup': 'nunique',
    'PayDelay': ['min', 'max', 'sum']
}).reset_index()

claims_agg.columns = ["Year", "MemberID", "ClaimsCount", "ProviderCount", "PrimaryConditionGroupCount", "MinPayDelay", "MaxPayDelay", "TotalPayDelay"]

claims_agg

,Year,MemberID,ClaimsCount,ProviderCount,PrimaryConditionGroupCount,MinPayDelay,MaxPayDelay,TotalPayDelay
0,1,210,4,3,4,22,162,327
1,1,3197,2,2,2,42,162,204
2,1,3889,5,5,5,36,156,316
3,1,4187,3,2,3,30,162,311
4,1,9063,1,1,1,67,67,67
...,...,...,...,...,...,...,...,...
218410,3,99987030,3,2,3,0,0,0
218411,3,99992565,2,1,2,24,30,54
218412,3,99993334,1,1,1,22,22,22
218413,3,99995391,11,6,11,0,65,368


In [88]:
claims_agg.loc[claims_agg['MemberID']==210]

,Year,MemberID,ClaimsCount,ProviderCount,PrimaryConditionGroupCount,MinPayDelay,MaxPayDelay,TotalPayDelay
0,1,210,4,3,4,22,162,327
76039,2,210,2,2,2,31,38,69
147473,3,210,2,2,2,23,41,64


Starting the analysis by looking at a correlation matrix between the different features

In [89]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(25, 15))
sns.set(font_scale=1.5)
sns.heatmap(df.corr(), annot=True)

NameError: name 'df' is not defined

<Figure size 2500x1500 with 0 Axes>

Next we are loading in the days in hospital data

In [112]:
days_y2 = pd.read_csv("HeritageHealth/HHP_release3/DaysInHospital_Y2.csv")
days_y2["Year"] = 2
days_y2

,MemberID,ClaimsTruncated,DaysInHospital,Year
0,24027423,0,0,2
1,98324177,0,0,2
2,33899367,1,1,2
3,5481382,0,1,2
4,69908334,0,0,2
...,...,...,...,...
76033,1702326,0,0,2
76034,31530440,0,0,2
76035,87102780,0,0,2
76036,74032946,0,0,2


In [111]:
days_y3 = pd.read_csv("HeritageHealth/HHP_release3/DaysInHospital_Y3.csv")
days_y3["Year"] = 3
days_y3

,MemberID,ClaimsTruncated,DaysInHospital,Year
0,90963501,0,0,3
1,85160905,0,0,3
2,45496648,0,0,3
3,45162557,0,0,3
4,68793937,0,0,3
...,...,...,...,...
71430,206785,0,0,3
71431,93465383,0,0,3
71432,57466586,0,0,3
71433,57283229,0,0,3


In [114]:
days = pd.concat([days_y2, days_y3])
days

,MemberID,ClaimsTruncated,DaysInHospital,Year
0,24027423,0,0,2
1,98324177,0,0,2
2,33899367,1,1,2
3,5481382,0,1,2
4,69908334,0,0,2
...,...,...,...,...
71430,206785,0,0,3
71431,93465383,0,0,3
71432,57466586,0,0,3
71433,57283229,0,0,3


We are also looking at the members data. This table consists of three feautres: MemberID, AgeAtFirstClaim and Sex.

In [90]:
members = pd.read_csv("HeritageHealth/HHP_release3/Members.csv")
members.isna().sum()

MemberID               0
AgeAtFirstClaim     5753
Sex                17552
dtype: int64

What we can see it that we have many missing values for both AgeAtFirstClaim and Sex. 

In [91]:
ageGroups = {"0-9": 0, "10-19": 10, "20-29": 20, "30-39": 30, "40-49": 40, "50-59": 50, 
        "60-69": 60, "70-79": 70, "80+": 80}

members["AgeAtFirstClaim"] = members["AgeAtFirstClaim"].replace(to_replace=ageGroups)
members["AgeAtFirstClaim"] = members["AgeAtFirstClaim"] .fillna(-1)

members["Sex"] = members["Sex"].fillna("N")

members

,MemberID,AgeAtFirstClaim,Sex
0,14723353,70.0,M
1,75706636,70.0,M
2,17320609,70.0,M
3,69690888,40.0,M
4,33004608,0.0,M
...,...,...,...
112995,99711514,40.0,F
112996,31690877,50.0,F
112997,9519985,30.0,F
112998,92806272,50.0,F


Processing the labcount data

In [115]:
labs = pd.read_csv("HeritageHealth/HHP_release3/LabCount.csv")
labs["DSFS"] = labs["DSFS"].replace(to_replace=dfds)
labs["DSFS"] = labs["DSFS"].fillna(0)
labs["LabCount"] = labs["LabCount"].replace(to_replace={"10+": "10"})
labs["Year"] = labs["Year"].replace(to_replace=year)

,MemberID,Year,DSFS,LabCount
0,69258001,3,3,1
1,10143167,1,1,2
2,1054357,1,1,6
3,56583841,3,7,4
4,70967047,2,1,2
...,...,...,...,...
361479,43366611,1,6,2
361480,72106973,3,4,10
361481,97884558,1,1,4
361482,38125799,3,7,3


Processing the drugcount data

In [134]:
drugs = pd.read_csv("HeritageHealth/HHP_release3/DrugCount.csv")
drugs["DSFS"] = drugs["DSFS"].replace(to_replace=dfds)
drugs["DSFS"] = drugs["DSFS"].fillna(0)
drugs["Year"] = drugs["Year"].replace(to_replace=year)
drugs["DrugCount"] = drugs["DrugCount"].replace(to_replace={"7+": "7"})

drugs = drugs.groupby(['Year', 'MemberID']).agg({'DrugCount': ['count']}).reset_index()
drugs.columns = ["Year", "MemberID", "DrugCount"]

### Merging the data

In [136]:
df = claims_agg.merge(members, how='left', on='MemberID')
df = df.merge(labs[["MemberID", "Year", "LabCount"]], how='left', on=['MemberID', 'Year'])
df = df.merge(drugs[["MemberID", "Year", "DrugCount"]], how='left', on=['MemberID', 'Year'])

df

,Year,MemberID,ClaimsCount,ProviderCount,PrimaryConditionGroupCount,MinPayDelay,MaxPayDelay,TotalPayDelay,AgeAtFirstClaim,Sex,LabCount,DrugCount
0,1,210,4,3,4,22,162,327,30.0,N,2,3.0
1,1,3197,2,2,2,42,162,204,0.0,F,NaN,4.0
2,1,3889,5,5,5,36,156,316,-1.0,F,10,10.0
3,1,4187,3,2,3,30,162,311,50.0,F,NaN,10.0
4,1,9063,1,1,1,67,67,67,60.0,F,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
424960,3,99995391,11,6,11,0,65,368,60.0,N,1,12.0
424961,3,99995391,11,6,11,0,65,368,60.0,N,2,12.0
424962,3,99995391,11,6,11,0,65,368,60.0,N,1,12.0
424963,3,99995391,11,6,11,0,65,368,60.0,N,4,12.0


## Training model

Splitting the data into training and test data

In [94]:
# split a dataset into train and test sets
from sklearn.model_selection import train_test_split

# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(claims, claims["PayDelay"], test_size=0.33)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(566213, 14) (278882, 14) (566213,) (278882,)


Creating a simple linear model

In [95]:
from sklearn.linear_model import LinearRegression

# Splitting the data into training and testing data
regr = LinearRegression()
  
regr.fit(X_train, y_train)

print(regr.score(X_test, y_test))

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values